# Sales Category Prediction - Incremental Learning Experiment

In [1]:
# imports
import numpy as np
import pandas as pd
import gc
import os
import re
import time
import psutil
import logging
import warnings
from datetime import datetime, timedelta
from tqdm.notebook import tqdm, trange
from glob import glob
import scipy.sparse

# Feature processing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import category_encoders as ce

# Models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore')

## Configuration

In [2]:
CONFIG = {
    # Paths
    "splits_dir": "../data/splits/",
    "weather_path": "../data/weather_processed.csv",
    "artifacts_dir": "../artifacts/",
    
    # Preprocessing
    "chunk_months": 1,  # Size of sliding window in months (1 or 2)
    "drop_columns": {"group", "year", "unit"},
    
    # Models
    "target_column": "qnt",
    "date_column": "calday",
    
    # Random seed
    "random_seed": 42
}

# Create artifacts directory if it doesn't exist
os.makedirs(CONFIG["artifacts_dir"], exist_ok=True)

In [3]:
# Feature categories
BOOL_EXPLICIT = {"bu_exists", "freezing_day", "cold_day", "warm_day", "hot_day"}
BOOL_PATTERNS = [r"^is_", r"^has_", r"_had_high_", r"_had_low_", r"_exists$"]

LOW_EXPLICIT = {
    "matrix_type", "country_id", "format_merch", "geolocal_type",
    "season", "type_bonus_id", "seasonal_group",
    "category_major", "category_detailed", "category_full"
}
LOW_PATTERNS = [r"^category_"]

MED_EXPLICIT = {
    "brand_id", "index_material", "index_store", "type_for_customer",
    "week_iso", "day_of_week", "day_of_month", "month", "quarter", "source_month"
}

NUM_PATTERNS = [
    r"^(temp|min|max|tempmax|tempmin|feelslike[a-z]*|dew|humidity|precip|snow|snowdepth"
    r"|wind(gust|speed|dir)|sealevelpressure|cloudcover|visibility"
    r"|solarradiation|solarenergy|uvindex|moonphase|daylight_hours"
    r"|heat_index|temp_range)",
    r".*_lag_\\d+d$", r".*_d\\d+to\\d+_(mean|min|max|std)$"
]

## Utility Functions

In [4]:
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024

def log_step(step_name):
    """Log step name and memory usage"""
    mem_mb = get_memory_usage()
    logging.info(f"{step_name}: {mem_mb:.2f} MB")
    return mem_mb

# Evaluation metrics
def calculate_mae(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def calculate_mape(y_true, y_pred):
    mask_nonzero = (y_true != 0) & (y_pred != 0)
    mask_one_zero = ((y_true == 0) & (y_pred != 0)) | ((y_true != 0) & (y_pred == 0))
    
    mape_nonzero = np.abs((y_true[mask_nonzero] - y_pred[mask_nonzero]) / y_true[mask_nonzero])
    mape_one_zero = np.ones(mask_one_zero.sum())
    
    if len(mape_nonzero) + len(mape_one_zero) == 0:
        return 0
    
    total_mape = np.concatenate([mape_nonzero, mape_one_zero]) if len(mape_one_zero) > 0 and len(mape_nonzero) > 0 else \
                (mape_nonzero if len(mape_nonzero) > 0 else mape_one_zero)
    return np.mean(total_mape)

def calculate_smape(y_true, y_pred):
    mask_nonzero = (y_true != 0) & (y_pred != 0)
    mask_one_zero = ((y_true == 0) & (y_pred != 0)) | ((y_true != 0) & (y_pred == 0))
    
    smape_nonzero = 2 * np.abs(y_true[mask_nonzero] - y_pred[mask_nonzero]) / (np.abs(y_true[mask_nonzero]) + np.abs(y_pred[mask_nonzero]))
    smape_one_zero = np.ones(mask_one_zero.sum())
    
    if len(smape_nonzero) + len(smape_one_zero) == 0:
        return 0
    
    total_smape = np.concatenate([smape_nonzero, smape_one_zero]) if len(smape_one_zero) > 0 and len(smape_nonzero) > 0 else \
                 (smape_nonzero if len(smape_nonzero) > 0 else smape_one_zero)
    return np.mean(total_smape)

# Additional metrics
def calculate_rmse(y_true, y_pred):
    """Calculate Root Mean Squared Error"""
    return np.sqrt(np.mean((y_true - y_pred) ** 2))

def calculate_r2(y_true, y_pred):
    """Calculate R-squared (coefficient of determination)"""
    from sklearn.metrics import r2_score
    return r2_score(y_true, y_pred)

def calculate_wape(y_true, y_pred):
    """Calculate Weighted Absolute Percentage Error"""
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

def calculate_bias(y_true, y_pred):
    """Calculate bias (Mean Percent Forecast Error)
    >0 → over-forecast (over-stock)
    <0 → under-forecast (under-stock)"""
    mask = y_true != 0
    if not np.any(mask):
        return 0
    
    percent_errors = (y_pred[mask] - y_true[mask]) / y_true[mask]
    return np.mean(percent_errors)  # Changed from median to mean

def calculate_stock_stats(y_true, y_pred, action_price):
    """Calculate over-stock and under-stock statistics"""
    differences = y_pred - y_true
    over_stock = np.where(differences > 0, differences * action_price, 0)
    under_stock = np.where(differences < 0, -differences * action_price, 0)
    
    total_sales = np.sum(y_true * action_price)
    over_stock_sum = np.sum(over_stock)
    under_stock_sum = np.sum(under_stock)
    
    # Avoid division by zero
    if total_sales == 0:
        over_stock_percent = 0
        under_stock_percent = 0
    else:
        over_stock_percent = over_stock_sum / total_sales * 100
        under_stock_percent = under_stock_sum / total_sales * 100
    
    return {
        'total_sales': total_sales,
        'over_stock': over_stock_sum,
        'under_stock': under_stock_sum,
        'over_stock_percent': over_stock_percent,
        'under_stock_percent': under_stock_percent
    }

In [5]:
def postprocess_predictions(y_pred, qnt_max):
    """Apply post-processing rules to prediction"""
    # Clip negative values to 0
    y_pred = np.clip(y_pred, 0, None)
    
    # Apply outlier correction where needed
    mask = (qnt_max >= 5) & (y_pred > 2 * qnt_max)
    if np.any(mask):
        y_pred[mask] = 2 * qnt_max[mask]
    
    # Round to nearest integer
    y_pred = np.round(y_pred).astype(int)
    
    return y_pred

## Data Preprocessing

In [6]:
def get_feature_types(df):
    """Identify feature types based on rules"""
    columns = df.columns.tolist()
    feature_types = {'bool_cols': [], 'low_cat_cols': [], 'med_cat_cols': [], 'num_cols': []}
    
    # Find target and ID columns
    target_col = CONFIG['target_column']
    date_col = CONFIG['date_column']
    drop_cols = list(CONFIG['drop_columns']) + [target_col, date_col] if target_col in columns and date_col in columns else []
    
    # Collect columns by type
    for col in columns:
        if col in drop_cols:
            continue
        
        # Check bool columns
        if col in BOOL_EXPLICIT or any(re.match(pattern, col) for pattern in BOOL_PATTERNS) or df[col].dtype == bool:
            feature_types['bool_cols'].append(col)
            continue
            
        # Check low cardinality categorical
        if col in LOW_EXPLICIT or any(re.match(pattern, col) for pattern in LOW_PATTERNS):
            feature_types['low_cat_cols'].append(col)
            continue
            
        # Check medium cardinality categorical
        if col in MED_EXPLICIT:
            feature_types['med_cat_cols'].append(col)
            continue
            
        # Check numeric by pattern
        if any(re.match(pattern, col) for pattern in NUM_PATTERNS) or df[col].dtype in ['int32', 'int64', 'float32', 'float64']:
            feature_types['num_cols'].append(col)
    
    return feature_types

def create_column_transformer(df):
    """Create a ColumnTransformer for preprocessing"""
    feature_types = get_feature_types(df)
    
    # Log the number of features of each type for debugging
    logging.info(f"Feature counts: bool={len(feature_types['bool_cols'])}, low_cat={len(feature_types['low_cat_cols'])}, "
                 f"med_cat={len(feature_types['med_cat_cols'])}, num={len(feature_types['num_cols'])}")
    
    transformers = []
    
    # Boolean columns to uint8
    if feature_types['bool_cols']:
        transformers.append(('bool', 'passthrough', feature_types['bool_cols']))
    
    # Low cardinality categorical columns to one-hot
    if feature_types['low_cat_cols']:
        transformers.append(('low_cat', 
                            OneHotEncoder(sparse_output=True, handle_unknown='ignore', dtype=np.int8),
                            feature_types['low_cat_cols']))
    
    # Medium cardinality categorical columns with count encoding
    if feature_types['med_cat_cols']:
        # First check if we have any medium cardinality columns
        if len(feature_types['med_cat_cols']) > 0:
            # Create a copy of the DataFrame with only the medium cardinality columns
            med_cat_df = df[feature_types['med_cat_cols']].copy()
            
            # Convert all columns to categorical type
            for col in med_cat_df.columns:
                med_cat_df[col] = med_cat_df[col].astype('category')
            transformers.append(('med_cat', 
                              ce.CountEncoder(normalize=True), 
                              feature_types['med_cat_cols']))
    
    # Numeric columns - just pass through, no imputation needed
    if feature_types['num_cols']:
        transformers.append(('num', 
                            'passthrough', 
                            feature_types['num_cols']))
    
    return ColumnTransformer(transformers, remainder='drop', n_jobs=1)  # Use single thread to reduce memory

def cast_types(df):
    """Downcast types for efficiency"""
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        elif df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
        elif df[col].dtype == 'bool':
            df[col] = df[col].astype('uint8')
    return df
    
def load_and_preprocess_data(split_file):
    """Load and preprocess a split file"""
    log_step(f"Loading split file {os.path.basename(split_file)}")
    
    # Read the split file
    df = pd.read_csv(split_file)
    
    # Read weather data
    weather_df = pd.read_csv(CONFIG['weather_path'])
    
    # Merge with weather data
    df = pd.merge(df, weather_df, on=CONFIG['date_column'], how='left')
    
    # Cast types
    df = cast_types(df)
    
    # Drop unnecessary columns
    for col in CONFIG['drop_columns']:
        if col in df.columns:
            df.drop(columns=[col], inplace=True)
    
    # Sort by date
    df.sort_values(by=CONFIG['date_column'], inplace=True)
    
    # Convert date to datetime
    if df[CONFIG['date_column']].dtype == 'object':
        df[CONFIG['date_column']] = pd.to_datetime(df[CONFIG['date_column']])
    
    # Explicitly convert medium cardinality columns to category without logging each conversion
    for col in MED_EXPLICIT:
        if col in df.columns:
            df[col] = df[col].astype('category')
    
    # Calculate max_qnt for post-processing only if it doesn't exist
    if 'qnt_max' not in df.columns:
        logging.info(f"'qnt_max' column not found, calculating it now")
        df['qnt_max'] = df.groupby(['brand_id', 'country_id', 'category_detailed'])[CONFIG['target_column']].transform('max')
    
    # Log transformation for specific quantity columns
    log_columns = [
        'qnt', 'qnt_loss', 'qnt_lag_14d', 'qnt_lag_21d', 'qnt_lag_28d', 
        'qnt_lag_avg', 'qnt_max', 'qnt_min', 'qnt_mean', 'qnt_median'
    ]
    
    # Store original values before transformation
    for col in log_columns:
        if col in df.columns:
            df[f"{col}_orig"] = df[col].copy()
    
    # Apply log1p transformation
    for col in log_columns:
        if col in df.columns:
            df[col] = np.log1p(df[col])
    
    log_step(f"Preprocessed split shape: {df.shape}")
    return df


## Model Training and Evaluation

In [7]:
def create_models():
    """Create models with optimized GPU acceleration parameters for fair comparison"""
    models = {
        "XGBoost": XGBRegressor(
            n_estimators=10000,
            learning_rate=0.05,
            max_depth=8,
            subsample=0.8,
            colsample_bytree=0.8,
            tree_method="gpu_hist",  # GPU acceleration
            predictor="gpu_predictor",
            max_bin=256,  # Optimize for limited VRAM
            objective="reg:squarederror",
            eval_metric="mae",  # Consistent metric across models
            n_jobs=-1,
            random_state=CONFIG["random_seed"],
            early_stopping_rounds=200  # Increased for better convergence
        ),
        "LightGBM": LGBMRegressor(
            device_type="gpu",  # GPU acceleration
            boosting_type="gbdt",
            n_estimators=10000,
            learning_rate=0.05,
            num_leaves=255,  # Equivalent to max_depth=8
            subsample=0.8,
            colsample_bytree=0.8,
            early_stopping_round=200,  # Correct parameter name (without 's')
            metric="mae",  # Consistent metric across models
            n_jobs=1,      # Use 1 for GPU mode
            gpu_platform_id=0,  # First OpenCL platform
            gpu_device_id=0,    # First GPU device
            verbose=-1,         # Progress info without too many messages
            random_state=CONFIG["random_seed"]
        ),
        "CatBoost": CatBoostRegressor(
            iterations=10000,
            depth=8,
            learning_rate=0.05,
            loss_function="RMSE",  # Use RMSE for GPU compatibility
            # Removed eval_metric="MAE" as it's not GPU compatible
            task_type="GPU",
            devices="0",  # Explicitly use first GPU
            random_seed=CONFIG["random_seed"],
            verbose=100,  # Show progress every 100 iterations
            early_stopping_rounds=200,
            allow_writing_files=False  # Save time by not writing snapshots
        )
    }
    
    return models

In [8]:
def train_and_evaluate_models(df, split_name):
    """Train models on the entire split without incremental learning"""
    log_step(f"Starting model training for split {split_name}")
    
    # Define target and date columns
    target_col = CONFIG['target_column']  # 'qnt' (log-transformed)
    target_orig_col = f"{target_col}_orig"  # Original target for evaluation
    date_col = CONFIG['date_column']
    qnt_max_col = 'qnt_max'
    qnt_max_orig_col = 'qnt_max_orig'
    
    # Exclude target column and date column from features
    feature_cols = [col for col in df.columns if col not in [target_col, target_orig_col, date_col] and not col.endswith('_orig')]
    
    # Create preprocessor
    preprocessor = create_column_transformer(df[feature_cols])
    
    # Create split for training (80%) and validation (20%)
    df_sorted = df.sort_values(by=date_col)
    split_idx = int(len(df_sorted) * 0.8)
    
    train_df = df_sorted.iloc[:split_idx]
    valid_df = df_sorted.iloc[split_idx:]
    
    # Extract features and target
    X_train = train_df[feature_cols]
    y_train = train_df[target_col]  # Log-transformed target
    
    X_valid = valid_df[feature_cols]
    y_valid = valid_df[target_col]  # Log-transformed target
    y_valid_orig = valid_df[target_orig_col]  # Original target for evaluation
    qnt_max_valid = valid_df[qnt_max_col]
    qnt_max_valid_orig = valid_df[qnt_max_orig_col]
    
    # Get action prices for stock statistics
    action_price = valid_df['action_price'] if 'action_price' in valid_df.columns else np.ones(len(valid_df))
    
    log_step(f"Train size: {len(X_train)}, Validation size: {len(X_valid)}")
    
    # Preprocess features
    X_train_proc = preprocessor.fit_transform(X_train)
    X_valid_proc = preprocessor.transform(X_valid)
    
    # Clean up memory
    del X_train, X_valid
    gc.collect()
    mem_mb = get_memory_usage()
    logging.info(f'After preprocessing, memory usage: {mem_mb:.2f} MB')
    
    # Create models
    models = create_models()
    
    # Results dictionary with all metrics
    results = {
        "split": [],
        "model": [],
        "MAE": [],
        "MAPE": [],
        "sMAPE": [],
        "RMSE": [],
        "R2": [],
        "WAPE": [],
        "Bias": [],
        "over_stock": [],
        "under_stock": [],
        "over_stock_percent": [],
        "under_stock_percent": [],
        "total_sales": [],
        "train_time_sec": [],
        "rows_train": [],
        "rows_valid": []
    }
    
    # Train and evaluate each model
    for model_name, model in models.items():
        log_step(f"Training {model_name}")
        start_time = time.time()
        
        if model_name == "CatBoost":
            # Import Pool for CatBoost
            from catboost import Pool
            
            # For CatBoost, we need dense matrices
            if scipy.sparse.issparse(X_train_proc):
                logging.info(f"Converting sparse matrix to dense for CatBoost GPU training")
                X_train_dense = X_train_proc.toarray().astype(np.float32)
                X_valid_dense = X_valid_proc.toarray().astype(np.float32)
            else:
                X_train_dense = X_train_proc
                X_valid_dense = X_valid_proc
            
            # Create Pool objects for CatBoost (recommended for better performance)
            train_pool = Pool(X_train_dense, y_train)
            valid_pool = Pool(X_valid_dense, y_valid)
            
            # Train CatBoost using Pool objects
            logging.info(f"Starting CatBoost training with GPU acceleration...")
            model.fit(
                train_pool, 
                eval_set=valid_pool,
                use_best_model=True,
                verbose=100  # Show progress every 100 iterations
            )
            
            # Clean up temporary objects
            del train_pool, valid_pool
            if scipy.sparse.issparse(X_train_proc):
                del X_train_dense, X_valid_dense
            gc.collect()
            
            # Predict - need to convert to dense for prediction if sparse
            if scipy.sparse.issparse(X_valid_proc):
                y_pred_log = model.predict(X_valid_proc.toarray().astype(np.float32))
            else:
                y_pred_log = model.predict(X_valid_proc)
                
        elif model_name == "XGBoost":
            # Train XGBoost with consistent API
            logging.info(f"Starting XGBoost training with GPU acceleration...")
            model.fit(
                X_train_proc, y_train,
                eval_set=[(X_valid_proc, y_valid)],
                verbose=100  # Show evaluation every 100 iterations
            )
            y_pred_log = model.predict(X_valid_proc)
            
        elif model_name == "LightGBM":
            # Log LightGBM GPU status without lgb.basic
            import lightgbm as lgb
            logging.info(f"LightGBM version: {lgb.__version__}")
            
            # Just log GPU settings from the model parameters
            logging.info(f"LightGBM GPU settings: device_type={model.device_type}, "
                         f"gpu_platform_id={model.gpu_platform_id}, gpu_device_id={model.gpu_device_id}")
            
            # Train LightGBM with consistent API and progress logging
            logging.info(f"Starting LightGBM training with GPU acceleration...")
            model.fit(
                X_train_proc, y_train,
                eval_set=[(X_valid_proc, y_valid)],
                callbacks=[lgb.log_evaluation(period=100)]  # Log every 100 iterations
            )
            y_pred_log = model.predict(X_valid_proc)
        
        # Verify log-scale prediction values
        logging.info(f"{model_name} - Log-scale predictions: min={y_pred_log.min():.4f}, max={y_pred_log.max():.4f}, mean={y_pred_log.mean():.4f}")
        
        # Transform back to original scale
        y_pred = np.expm1(y_pred_log)
        
        # Apply post-processing (includes rounding to integers) using original-scale qnt_max
        y_pred = postprocess_predictions(y_pred, qnt_max_valid_orig.values)
        
        # Verify original-scale predictions after postprocessing
        logging.info(f"{model_name} - Original scale predictions: min={y_pred.min():.4f}, max={y_pred.max():.4f}, mean={y_pred.mean():.4f}")
        
        # Calculate metrics using original-scale values
        mae = calculate_mae(y_valid_orig.values, y_pred)
        mape = calculate_mape(y_valid_orig.values, y_pred)
        smape = calculate_smape(y_valid_orig.values, y_pred)
        
        # Calculate additional metrics on original scale
        rmse = calculate_rmse(y_valid_orig.values, y_pred)
        r2 = calculate_r2(y_valid_orig.values, y_pred)
        wape = calculate_wape(y_valid_orig.values, y_pred)
        bias = calculate_bias(y_valid_orig.values, y_pred)
        
        # Calculate stock statistics on original scale
        stock_stats = calculate_stock_stats(y_valid_orig.values, y_pred, action_price)
        
        # Record training time
        train_time = time.time() - start_time
        
        # Log detailed results
        logging.info(f"{model_name} - Training time: {train_time:.2f}s")
        logging.info(f"{model_name} - MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
        logging.info(f"{model_name} - MAPE: {mape:.4f}, sMAPE: {smape:.4f}, WAPE: {wape:.4f}, Bias: {bias:.4f}")
        logging.info(f"{model_name} - Over-stock: {stock_stats['over_stock']:.2f} ({stock_stats['over_stock_percent']:.2f}%), Under-stock: {stock_stats['under_stock']:.2f} ({stock_stats['under_stock_percent']:.2f}%)")
        
        # Save results
        results["split"].append(split_name)
        results["model"].append(model_name)
        results["MAE"].append(mae)
        results["MAPE"].append(mape)
        results["sMAPE"].append(smape)
        results["RMSE"].append(rmse)
        results["R2"].append(r2)
        results["WAPE"].append(wape)
        results["Bias"].append(bias)
        results["over_stock"].append(stock_stats['over_stock'])
        results["under_stock"].append(stock_stats['under_stock'])
        results["over_stock_percent"].append(stock_stats['over_stock_percent'])
        results["under_stock_percent"].append(stock_stats['under_stock_percent'])
        results["total_sales"].append(stock_stats['total_sales'])
        results["train_time_sec"].append(train_time)
        results["rows_train"].append(len(y_train))
        results["rows_valid"].append(len(y_valid))
        
        # Save model
        model_filename = f"{CONFIG['artifacts_dir']}{split_name}_{model_name}.model"
        if model_name == "XGBoost":
            model.save_model(model_filename)
        elif model_name == "LightGBM":
            model.booster_.save_model(model_filename)
        elif model_name == "CatBoost":
            model.save_model(model_filename)
    
    # Clean up memory
    del y_train, y_valid, y_valid_orig, X_train_proc, X_valid_proc
    gc.collect()
    
    return pd.DataFrame(results)

## Main Execution

In [9]:
# Find all split files
split_files = glob(CONFIG['splits_dir'] + 'sales_category_*.csv')

# Initialize results tracking
all_results = []

# Process each split file
for split_file in tqdm(split_files, desc="Processing splits"):
    split_name = os.path.basename(split_file).replace('sales_category_', '').replace('.csv', '')
    
    log_step(f"Starting processing of split {split_name}")
    
    # Load and preprocess data
    df = load_and_preprocess_data(split_file)
    
    # Train and evaluate models
    split_results = train_and_evaluate_models(df, split_name)
    all_results.append(split_results)
    
    # Clean up to free memory
    del df, split_results
    gc.collect()
    mem_mb = get_memory_usage()
    logging.info(f'After split {split_name} completion - memory: {mem_mb} MB')

# Combine results
results_df = pd.concat(all_results)

# Calculate averages across splits for each model, including ALL metrics
avg_results = results_df.groupby('model').agg({
    'MAE': 'mean',
    'MAPE': 'mean', 
    'sMAPE': 'mean',
    'RMSE': 'mean',
    'R2': 'mean',
    'WAPE': 'mean',
    'Bias': 'mean',
    'over_stock': 'mean',
    'under_stock': 'mean',
    'over_stock_percent': 'mean',
    'under_stock_percent': 'mean',
    'total_sales': 'mean',
    'train_time_sec': 'mean',
    'rows_train': 'mean',
    'rows_valid': 'mean'
}).reset_index()
avg_results['split'] = 'AVERAGE'

# Add averages to results
final_results = pd.concat([results_df, avg_results])

# Save results
final_results.to_csv(f"{CONFIG['artifacts_dir']}experiment_metrics.csv", index=False)

# Display results
print("\nFinal Results:")
print("\nPrimary Metrics:")
display(final_results[['split', 'model', 'MAE', 'RMSE', 'R2', 'sMAPE', 'WAPE']])

print("\nStock Impact Metrics:")
display(final_results[['split', 'model', 'over_stock_percent', 'under_stock_percent', 'Bias']])

print("\nDetailed Metrics:")
display(final_results)

Processing splits:   0%|          | 0/11 [00:00<?, ?it/s]

2025-05-16 00:58:16,671 - root - INFO - Starting processing of split 01_4202: 232.85 MB
2025-05-16 00:58:16,671 - root - INFO - Loading split file sales_category_01_4202.csv: 232.85 MB
2025-05-16 01:06:44,169 - root - INFO - Preprocessed split shape: (7543435, 521): 13997.55 MB
2025-05-16 01:06:44,201 - root - INFO - Starting model training for split 01_4202: 14000.23 MB
2025-05-16 01:07:16,268 - root - INFO - Feature counts: bool=20, low_cat=9, med_cat=7, num=471
2025-05-16 01:09:10,677 - root - INFO - Train size: 6034748, Validation size: 1508687: 25045.09 MB
2025-05-16 01:16:38,649 - root - INFO - After preprocessing, memory usage: 6612.15 MB
2025-05-16 01:16:38,667 - root - INFO - Training XGBoost: 6613.14 MB
2025-05-16 01:16:38,668 - root - INFO - Starting XGBoost training with GPU acceleration...


[0]	validation_0-mae:0.76040
[100]	validation_0-mae:0.34493
[200]	validation_0-mae:0.31538
[300]	validation_0-mae:0.30548
[400]	validation_0-mae:0.29692
[500]	validation_0-mae:0.29243
[600]	validation_0-mae:0.28881
[700]	validation_0-mae:0.28580
[800]	validation_0-mae:0.28371
[900]	validation_0-mae:0.28223
[1000]	validation_0-mae:0.28047
[1100]	validation_0-mae:0.27953
[1200]	validation_0-mae:0.27891
[1300]	validation_0-mae:0.27858
[1400]	validation_0-mae:0.27822
[1500]	validation_0-mae:0.27778
[1600]	validation_0-mae:0.27735
[1700]	validation_0-mae:0.27707
[1800]	validation_0-mae:0.27703
[1900]	validation_0-mae:0.27680
[2000]	validation_0-mae:0.27633
[2100]	validation_0-mae:0.27602
[2200]	validation_0-mae:0.27592
[2300]	validation_0-mae:0.27569
[2400]	validation_0-mae:0.27535
[2500]	validation_0-mae:0.27552
[2582]	validation_0-mae:0.27552


2025-05-16 02:18:28,156 - root - INFO - XGBoost - Log-scale predictions: min=-2.0093, max=5.0842, mean=0.5224
2025-05-16 02:18:28,285 - root - INFO - XGBoost - Original scale predictions: min=0.0000, max=160.0000, mean=1.5638
2025-05-16 02:18:28,917 - root - INFO - XGBoost - Training time: 3710.25s
2025-05-16 02:18:28,920 - root - INFO - XGBoost - MAE: 1.7236, RMSE: 6.5815, R²: 0.3910
2025-05-16 02:18:28,921 - root - INFO - XGBoost - MAPE: 0.7063, sMAPE: 0.6671, WAPE: 0.6565, Bias: 0.0722
2025-05-16 02:18:28,922 - root - INFO - XGBoost - Over-stock: 16086145.66 (10.84%), Under-stock: 71296309.30 (48.05%)
2025-05-16 02:18:29,399 - root - INFO - Training LightGBM: 6455.72 MB
2025-05-16 02:18:29,403 - root - INFO - LightGBM version: 4.6.0
2025-05-16 02:18:29,406 - root - INFO - LightGBM GPU settings: device_type=gpu, gpu_platform_id=0, gpu_device_id=0
2025-05-16 02:18:29,408 - root - INFO - Starting LightGBM training with GPU acceleration...


[100]	valid_0's l1: 0.292898
[200]	valid_0's l1: 0.271289
[300]	valid_0's l1: 0.263957
[400]	valid_0's l1: 0.259713
[500]	valid_0's l1: 0.25719
[600]	valid_0's l1: 0.255946
[700]	valid_0's l1: 0.255195
[800]	valid_0's l1: 0.254114
[900]	valid_0's l1: 0.253293
[1000]	valid_0's l1: 0.253159
[1100]	valid_0's l1: 0.253026
[1200]	valid_0's l1: 0.253061
[1300]	valid_0's l1: 0.252462
[1400]	valid_0's l1: 0.252201
[1500]	valid_0's l1: 0.251959
[1600]	valid_0's l1: 0.251787
[1700]	valid_0's l1: 0.251724
[1800]	valid_0's l1: 0.251718
[1900]	valid_0's l1: 0.251585
[2000]	valid_0's l1: 0.251343
[2100]	valid_0's l1: 0.251382
[2200]	valid_0's l1: 0.251518


...

,split,model,MAE,RMSE,R2,sMAPE,WAPE
0,01_4202,XGBoost,1.723642,6.581519,0.390962,0.667067,0.656493
1,01_4202,LightGBM,1.651121,6.365900,0.430214,0.638503,0.628872
2,01_4202,CatBoost,1.729084,6.727494,0.363645,0.668455,0.658566
0,02_3801,XGBoost,0.464424,1.895414,0.538356,0.598043,0.599490
1,02_3801,LightGBM,0.454005,1.881362,0.545176,0.585199,0.586040
2,02_3801,CatBoost,0.460659,1.955350,0.508699,0.590414,0.594629
0,03_2314,XGBoost,0.582227,1.947627,0.500805,0.543018,0.570594
1,03_2314,LightGBM,0.566969,1.889504,0.530155,0.529324,0.555640
2,03_2314,CatBoost,0.591127,1.957850,0.495551,0.552663,0.579316
0,04_3501,XGBoost,0.654328,2.245877,0.583463,0.536874,0.543415



Stock Impact Metrics:


,split,model,over_stock_percent,under_stock_percent,Bias
0,01_4202,XGBoost,10.840388,48.046293,0.072175
1,01_4202,LightGBM,12.496775,42.530986,0.131858
2,01_4202,CatBoost,7.602723,51.872627,-0.035442
0,02_3801,XGBoost,18.754795,42.448607,0.115788
1,02_3801,LightGBM,17.858321,41.879630,0.099257
2,02_3801,CatBoost,13.929098,46.625557,0.013045
0,03_2314,XGBoost,13.072413,40.011147,0.037892
1,03_2314,LightGBM,14.140778,37.173025,0.104172
2,03_2314,CatBoost,13.307453,40.341917,0.024983
0,04_3501,XGBoost,11.759696,41.867821,-0.019788



Detailed Metrics:


,split,model,MAE,MAPE,sMAPE,RMSE,R2,WAPE,Bias,over_stock,under_stock,over_stock_percent,under_stock_percent,total_sales,train_time_sec,rows_train,rows_valid
0,01_4202,XGBoost,1.723642,0.706311,0.667067,6.581519,0.390962,0.656493,0.072175,1.608615e+07,7.129631e+07,10.840388,48.046293,148390864.0,3710.249579,6.034748e+06,1.508687e+06
1,01_4202,LightGBM,1.651121,0.702801,0.638503,6.365900,0.430214,0.628872,0.131858,1.854407e+07,6.311210e+07,12.496775,42.530986,148390864.0,4542.013697,6.034748e+06,1.508687e+06
2,01_4202,CatBoost,1.729084,0.652418,0.668455,6.727494,0.363645,0.658566,-0.035442,1.128175e+07,7.697424e+07,7.602723,51.872627,148390864.0,4692.665754,6.034748e+06,1.508687e+06
0,02_3801,XGBoost,0.464424,0.648761,0.598043,1.895414,0.538356,0.599490,0.115788,9.086616e+06,2.056616e+07,18.754795,42.448607,48449560.0,509.668415,4.945068e+06,1.236268e+06
1,02_3801,LightGBM,0.454005,0.629405,0.585199,1.881362,0.545176,0.586040,0.099257,8.652278e+06,2.029050e+07,17.858321,41.879630,48449560.0,2737.930416,4.945068e+06,1.236268e+06
2,02_3801,CatBoost,0.460659,0.597623,0.590414,1.955350,0.508699,0.594629,0.013045,6.748587e+06,2.258988e+07,13.929098,46.625557,48449560.0,3574.774239,4.945068e+06,1.236268e+06
0,03_2314,XGBoost,0.582227,0.559859,0.543018,1.947627,0.500805,0.570594,0.037892,1.440859e+07,4.410081e+07,13.072413,40.011147,110221320.0,849.669014,4.216239e+06,1.054060e+06
1,03_2314,LightGBM,0.566969,0.574746,0.529324,1.889504,0.530155,0.555640,0.104172,1.558615e+07,4.097260e+07,14.140778,37.173025,110221320.0,6648.928876,4.216239e+06,1.054060e+06
2,03_2314,CatBoost,0.591127,0.567536,0.552663,1.957850,0.495551,0.579316,0.024983,1.466765e+07,4.446539e+07,13.307453,40.341917,110221320.0,3592.023669,4.216239e+06,1.054060e+06
0,04_3501,XGBoost,0.654328,0.537010,0.536874,2.245877,0.583463,0.543415,-0.019788,9.167631e+06,3.263934e+07,11.759696,41.867821,77958056.0,361.658547,4.073541e+06,1.018386e+06


In [11]:
final_results.round(2)

,split,model,MAE,MAPE,sMAPE,RMSE,R2,WAPE,Bias,over_stock,under_stock,over_stock_percent,under_stock_percent,total_sales,train_time_sec,rows_train,rows_valid
0,01_4202,XGBoost,1.72,0.71,0.67,6.58,0.39,0.66,0.07,16086145.66,71296309.30,10.84,48.05,148390864.0,3710.25,6034748.00,1508687.00
1,01_4202,LightGBM,1.65,0.70,0.64,6.37,0.43,0.63,0.13,18544072.76,63112097.97,12.50,42.53,148390864.0,4542.01,6034748.00,1508687.00
2,01_4202,CatBoost,1.73,0.65,0.67,6.73,0.36,0.66,-0.04,11281746.35,76974239.59,7.60,51.87,148390864.0,4692.67,6034748.00,1508687.00
0,02_3801,XGBoost,0.46,0.65,0.60,1.90,0.54,0.60,0.12,9086615.66,20566163.41,18.75,42.45,48449560.0,509.67,4945068.00,1236268.00
1,02_3801,LightGBM,0.45,0.63,0.59,1.88,0.55,0.59,0.10,8652277.85,20290496.64,17.86,41.88,48449560.0,2737.93,4945068.00,1236268.00
2,02_3801,CatBoost,0.46,0.60,0.59,1.96,0.51,0.59,0.01,6748586.84,22589877.14,13.93,46.63,48449560.0,3574.77,4945068.00,1236268.00
0,03_2314,XGBoost,0.58,0.56,0.54,1.95,0.50,0.57,0.04,14408585.87,44100814.60,13.07,40.01,110221320.0,849.67,4216239.00,1054060.00
1,03_2314,LightGBM,0.57,0.57,0.53,1.89,0.53,0.56,0.10,15586151.72,40972599.39,14.14,37.17,110221320.0,6648.93,4216239.00,1054060.00
2,03_2314,CatBoost,0.59,0.57,0.55,1.96,0.50,0.58,0.02,14667650.26,44465393.02,13.31,40.34,110221320.0,3592.02,4216239.00,1054060.00
0,04_3501,XGBoost,0.65,0.54,0.54,2.25,0.58,0.54,-0.02,9167630.66,32639339.28,11.76,41.87,77958056.0,361.66,4073541.00,1018386.00


In [12]:
print("\nFinal Results:")
print("\nPrimary Metrics:")
display(final_results[['split', 'model', 'MAE', 'RMSE', 'R2', 'sMAPE', 'WAPE']])


Final Results:

Primary Metrics:


,split,model,MAE,RMSE,R2,sMAPE,WAPE
0,01_4202,XGBoost,1.723642,6.581519,0.390962,0.667067,0.656493
1,01_4202,LightGBM,1.651121,6.365900,0.430214,0.638503,0.628872
2,01_4202,CatBoost,1.729084,6.727494,0.363645,0.668455,0.658566
0,02_3801,XGBoost,0.464424,1.895414,0.538356,0.598043,0.599490
1,02_3801,LightGBM,0.454005,1.881362,0.545176,0.585199,0.586040
2,02_3801,CatBoost,0.460659,1.955350,0.508699,0.590414,0.594629
0,03_2314,XGBoost,0.582227,1.947627,0.500805,0.543018,0.570594
1,03_2314,LightGBM,0.566969,1.889504,0.530155,0.529324,0.555640
2,03_2314,CatBoost,0.591127,1.957850,0.495551,0.552663,0.579316
0,04_3501,XGBoost,0.654328,2.245877,0.583463,0.536874,0.543415


In [13]:
print("\nStock Impact Metrics:")
display(final_results[['split', 'model', 'over_stock_percent', 'under_stock_percent', 'Bias']])


Stock Impact Metrics:


,split,model,over_stock_percent,under_stock_percent,Bias
0,01_4202,XGBoost,10.840388,48.046293,0.072175
1,01_4202,LightGBM,12.496775,42.530986,0.131858
2,01_4202,CatBoost,7.602723,51.872627,-0.035442
0,02_3801,XGBoost,18.754795,42.448607,0.115788
1,02_3801,LightGBM,17.858321,41.879630,0.099257
2,02_3801,CatBoost,13.929098,46.625557,0.013045
0,03_2314,XGBoost,13.072413,40.011147,0.037892
1,03_2314,LightGBM,14.140778,37.173025,0.104172
2,03_2314,CatBoost,13.307453,40.341917,0.024983
0,04_3501,XGBoost,11.759696,41.867821,-0.019788
